In [ ]:
from ortools.linear_solver import pywraplp
K = 2

def ppsr(dist_matrix):
    n = len(dist_matrix)
    solver = pywraplp.Solver.CreateSolver('SCIP')
    # Decision Variables: X[i][j][k] = 1 nếu xe k đi từ i-j
    X = {}
    for i in range(n):
        for j in range(n):
            if i != j:
                for k in range(K):
                    X[i, j, k] = solver.BoolVar(f'X[{i},{j},{k}]')
                    
     # Total distance cho mỗi xe
    total_distance = [solver.IntVar(0, solver.infinity(), f'total_distance[{m}]') for m in range(K)]
    
    # Objective function
    max_distance = solver.IntVar(0, solver.infinity(), 'max_distance')
    
    solver.Add(max_distance == solver.Max(total_distance))
    solver.Minimize(max_distance)
    
    Y = {}
    for i in range(N):
        for k in range(K):
            Y[i, k] = solver.BoolVar(f'Y[{i},{k}]')
    Z = {}
    for i in range(M):
        for k in range(K):
            Z[i, k] = solver.BoolVar(f'Z[{i},{k}]')
            
    for i in range(N):
        solver.Add(sum(Y[i, k] for k in range(K)) == 1)
        for k in range(K):
            solver.Add(X[i, i + N + M, k]>=Y[i, k]) 
            
    for i in range(M):
        solver.Add(sum(Z[i, k] for k in range(K)) == 1)    
    for k in range(K):
        solver.Add(sum(q[i] * Z[i, k] for i in range(M)) <= Q[k])
    
        

In [20]:
from ortools.linear_solver import pywraplp

def main():
    solver = pywraplp.Solver.CreateSolver('SCIP')
    
    # Hardcoded input values
    N = 3  # Number of passenger requests
    M = 3  # Number of parcel requests
    K = 2  # Number of taxis

    # Parcel request quantities
    q = [8, 4, 5]

    # Taxi capacities
    Q = [16, 16]

    # Distance matrix
    d = [
        [0, 8, 7, 9, 6, 5, 11, 6, 11, 12, 12, 12, 13],
        [8, 0, 4, 1, 2, 8, 5, 13, 19, 12, 4, 8, 9],
        [7, 4, 0, 3, 3, 8, 4, 12, 15, 8, 5, 6, 7],
        [9, 1, 3, 0, 3, 9, 4, 14, 19, 11, 3, 7, 8],
        [6, 2, 3, 3, 0, 6, 6, 11, 17, 11, 6, 9, 10],
        [5, 8, 8, 9, 6, 0, 12, 5, 16, 15, 12, 15, 15],
        [11, 5, 4, 4, 6, 12, 0, 16, 18, 7, 4, 3, 4],
        [6, 13, 12, 14, 11, 5, 16, 0, 15, 18, 17, 18, 19],
        [11, 19, 15, 19, 17, 16, 18, 15, 0, 13, 21, 17, 17],
        [12, 12, 8, 11, 11, 15, 7, 18, 13, 0, 11, 5, 4],
        [12, 4, 5, 3, 6, 12, 4, 17, 21, 11, 0, 7, 8],
        [12, 8, 6, 7, 9, 15, 3, 18, 17, 5, 7, 0, 1],
        [13, 9, 7, 8, 10, 15, 4, 19, 17, 4, 8, 1, 0]
    ]

    # Decision Variables
    X = {}
    for i in range(2 * N + 2 * M + 1):
        for j in range(2 * N + 2 * M + 1):
            for k in range(K):
                X[i, j, k] = solver.BoolVar(f'X[{i},{j},{k}]')

    Y = {}
    for i in range(N):
        for k in range(K):
            Y[i, k] = solver.BoolVar(f'Y[{i},{k}]')

    Z = {}
    for i in range(M):
        for k in range(K):
            Z[i, k] = solver.BoolVar(f'Z[{i},{k}]')

    T = [solver.IntVar(0, solver.infinity(), f'T[{k}]') for k in range(K)]
    D = solver.IntVar(0, solver.infinity(), 'D')

    
    solver.Minimize(D)
    
    for k in range(K):
        solver.Add(T[k] <= D)

    # 2. Each passenger must be served by one and only one taxi, in a direct route.
 
    for i in range(N):
        solver.Add(sum(Y[i, k] for k in range(K)) == 1)
        for k in range(K):
            # If passenger i is served by taxi k, there should be a direct route from pickup to drop-off
            solver.Add(X[i, i + N + M, k]>=Y[i, k])  # Direct route for passenger
            

    # 3. Each parcel request should be handled by one and only one taxi.
    for i in range(M):
        solver.Add(sum(Z[i, k] for k in range(K)) == 1)

    # 4. Ensure taxi capacity constraints for parcels
    for k in range(K):
        solver.Add(sum(q[i] * Z[i, k] for i in range(M)) <= Q[k])

    # 5. Ensure each taxi returns to the depot (starting point)
    #for k in range(K):
        #solver.Add(X[0, 0, k] == 1)

    # 6. Calculate the total distance traveled by each taxi k
    for k in range(K):
        solver.Add(T[k] == sum(d[i][j] * X[i, j, k] for i in range(2 * N + 2 * M + 1) for j in range(2 * N + 2 * M + 1)))

    # Solve
    status = solver.Solve()

    # Output
    if status == pywraplp.Solver.OPTIMAL:
        print(solver.Objective().Value())  
        

if __name__ == "__main__":
    main()

24.000000000000004


In [29]:
from ortools.linear_solver import pywraplp

def solve_taxi_routing(N, M, K, q, Q, dist):
    # Create the solver
    solver = pywraplp.Solver.CreateSolver('SCIP')

    # Decision variables
    x = {}
    y = {}
    z = {}

    # x[k][i]: taxi k serves passenger i
    for k in range(K):
        for i in range(N):
            x[k, i] = solver.BoolVar(f'x[{k},{i}]')
    
    # y[k][j]: taxi k serves parcel j
    for k in range(K):
        for j in range(M):
            y[k, j] = solver.BoolVar(f'y[{k},{j}]')
    
    # z[k][i][j]: taxi k travels from i to j
    for k in range(K):
        for i in range(2 * N + 2 * M + 1):
            for j in range(2 * N + 2 * M + 1):
                z[k, i, j] = solver.BoolVar(f'z[{k},{i},{j}]')

    # Objective: minimize the maximum distance traveled by any taxi
    max_distance = solver.NumVar(0, solver.infinity(), 'max_distance')

    # Each taxi starts and ends at the depot (0)
    for k in range(K):
        # Outgoing from depot
        solver.Add(sum(z[k, 0, j] for j in range(1, 2 * N + 2 * M + 1)) == 1)
        # Incoming to depot
        solver.Add(sum(z[k, i, 0] for i in range(1, 2 * N + 2 * M + 1)) == 1)

    # Each passenger must be served exactly once
    for i in range(N):
        solver.Add(sum(x[k, i] for k in range(K)) == 1)

    # Each parcel must be served exactly once
    for j in range(M):
        solver.Add(sum(y[k, j] for k in range(K)) == 1)

    # Capacity constraints
    for k in range(K):
        solver.Add(sum(q[j] * y[k, j] for j in range(M)) <= Q[k])

    # Route constraints
    for k in range(K):
        for i in range(1, 2 * N + 2 * M + 1):
            # Ensure that each taxi travels to and from served locations
            incoming = sum(z[k, j, i] for j in range(2 * N + 2 * M + 1))
            outgoing = sum(z[k, i, j] for j in range(2 * N + 2 * M + 1))
            solver.Add(incoming == outgoing)  # Flow conservation

            for j in range(2 * N + 2 * M + 1):
                if i < N:  # passenger pickup
                    solver.Add(z[k, i, j] <= x[k, i])  # Ensure travel is only if served
                elif N <= i < N + M:  # parcel pickup
                    solver.Add(z[k, i, j] <= y[k, i - N])  # Ensure travel is only if served
                elif i >= N + M:  # passenger drop-off
                    passenger_idx = i - (N + M)
                    solver.Add(z[k, i, j] <= x[k, passenger_idx])  # Ensure travel is only if served

    # Add constraints for distance calculation
    for k in range(K):
        for i in range(2 * N + 2 * M + 1):
            for j in range(2 * N + 2 * M + 1):
                solver.Add(z[k, i, j] * dist[i][j] <= max_distance)

    # Objective function: minimize max_distance
    solver.Minimize(max_distance)

    # Solve the problem
    status = solver.Solve()

    # Output the results
    if status == pywraplp.Solver.OPTIMAL:
        print(K)
        for k in range(K):
            route = []
            for i in range(2 * N + 2 * M + 1):
                for j in range(2 * N + 2 * M + 1):
                    if z[k, i, j].solution_value() == 1:
                        route.append(i)
            print(len(route))
            print(' '.join(map(str, route)))
    else:
        print('No optimal solution found.')

# Example Input
N = 3
M = 3
K = 2
q = [8, 4, 5]
Q = [16, 16]
dist = [
    [0, 8, 7, 9, 6, 5, 11, 6, 11, 12, 12, 12, 13],
    [8, 0, 4, 1, 2, 8, 5, 13, 19, 12, 4, 8, 9],
    [7, 4, 0, 3, 3, 8, 4, 12, 15, 8, 5, 6, 7],
    [9, 1, 3, 0, 3, 9, 4, 14, 19, 11, 3, 7, 8],
    [6, 2, 3, 3, 0, 6, 6, 11, 17, 11, 6, 9, 10],
    [5, 8, 8, 9, 6, 0, 12, 5, 16, 15, 12, 15, 15],
    [11, 5, 4, 4, 6, 12, 0, 16, 18, 7, 4, 3, 4],
    [6, 13, 12, 14, 11, 5, 16, 0, 15, 18, 17, 18, 19],
    [11, 19, 15, 19, 17, 16, 18, 15, 0, 13, 21, 17, 17],
    [12, 12, 8, 11, 11, 15, 7, 18, 13, 0, 11, 5, 4],
    [12, 4, 5, 3, 6, 12, 4, 17, 21, 11, 0, 7, 8],
    [12, 8, 6, 7, 9, 15, 3, 18, 17, 5, 7, 0, 1],
    [13, 9, 7, 8, 10, 15, 4, 19, 17, 4, 8, 1, 0]
]

# Solve the taxi routing problem
solve_taxi_routing(N, M, K, q, Q, dist)

2
2
0 5
2
0 5
